In [83]:
import pandas as pd
import psycopg2

In [99]:
def create_workshops_list(row):
    if pd.notnull(row.iloc[6]):
        return list(range(len(row) - 8))
    else:
        return [i for i, workshop in enumerate(row[7:-1], start=0) if pd.notnull(workshop)]
    
seriesName = 'test'
rawSurvey = pd.read_csv('../productionScripts/qualtricsCSVs/DataLab Workshops Template.csv')
filteredSurvey = rawSurvey[rawSurvey.columns[18:]]
filteredSurvey = filteredSurvey.drop([0,1])
filteredSurvey.reset_index(inplace=True, drop=True)
filteredSurvey['Workshops'] = filteredSurvey.apply(create_workshops_list, axis=1)
filteredSurvey.drop(columns = filteredSurvey.columns[6:-2], inplace=True)
filteredSurvey["Recontact"] = filteredSurvey["Recontact"].fillna('No').map({'Yes': True, 'No': False})
filteredSurvey['Series'] = seriesName
UA = filteredSurvey.dropna(subset=["UAEmail"])
nonUA = filteredSurvey.dropna(subset=["NonUAEmail"])

UA = UA.reset_index(drop=True)
nonUA = nonUA.reset_index(drop=True)

UA_Filtered = UA[["UAEmail", "FirstName", "LastName", "Workshops", "Series", "Recontact"]]
nonUA_Filtered = nonUA[["NonUAEmail", "FirstName", "LastName", "Organization", "Workshops", "Series", "Recontact"]]

In [100]:
UA_Filtered

,UAEmail,FirstName,LastName,Workshops,Series,Recontact
0,austinmedina@arizona.edu,Austin,Medina,"[0, 2]",test,True
1,austinmedina@arizona.edu,Austin,Medina,"[0, 1, 2, 3]",test,True


In [102]:
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)

cur = conn.cursor()
cur.execute("""
                SELECT workshopID FROM workshops WHERE seriesID = %s
                """, (14,))
conn.commit()
workshops = cur.fetchall()
workshopsToAdd = [workshops[i][0] for i in UA_Filtered.loc[0]['Workshops']]
workshopsToAdd

[113, 115]

In [64]:


# filteredSurvey['Workshops'] = filteredSurvey.iloc[:, 6:-1].apply(lambda row: row.dropna().tolist(), axis=1)
# filteredSurvey.drop(columns = filteredSurvey.columns[6:-2], inplace=True)
# filteredSurvey = filteredSurvey.drop_duplicates(subset=["NonUAEmail", 'UAEmail'])
# filteredSurvey["Series"] = seriesName
# filteredSurvey["Recontact"] = filteredSurvey["Recontact"].fillna('No').map({'Yes': True, 'No': False})

In [68]:
filteredSurvey

,UA-Affiliated,FirstName,LastName,UAEmail,Organization,NonUAEmail,Workshops_1,Workshops_2,Workshops_3,Workshops_4,Workshops_5,Recontact,Workshops
0,Yes,Austin,Medina,austinmedina@arizona.edu,NaN,NaN,NaN,All of the sessions,NaN,All of the sessions,NaN,Yes,"[0, 2]"
1,Yes,Austin,Medina,austinmedina@arizona.edu,NaN,NaN,All of the sessions,NaN,NaN,NaN,NaN,Yes,"[0, 1, 2, 3]"
